# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import re
import nltk
import pickle
import numpy as np
import pandas as pd
nltk.download('stopwords')
nltk.download(['punkt', 'wordnet'])
from nltk.corpus import stopwords 
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from nltk.tokenize import word_tokenize, RegexpTokenizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
X = df['message']
y = df[df.columns[5:]]

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    """
    Behaviour: Split text into words and return the root form of the words
    Args:
      text(str): text data.
    Return:
      clean_tokens(list of str): List of tokens extracted from the provided text
    """
    # Normalize text:Convert to lowercase and Remove punctuation
    text = text.lower() 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)
    
    # Tokenize text:Split text into words using NLTK
    words = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in words if t not in stop]
    
   # lemmatize as shown in the lesson
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(w) for w in words]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
# Create a instance for RandomFrorestClassifier()
estimator_rf = MultiOutputClassifier(RandomForestClassifier())

pipeline_rf = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', estimator_rf)
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
# Train-test splitting
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_rf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
# Get results and add them to a dataframe.
def print_results(y_test, y_pred):
    # declare an empty dataframe to store the results
    results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    num = 0
    for col in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[col], y_pred[:,num], average='weighted')
        results.set_value(num+1, 'Category', col)
        results.set_value(num+1, 'f_score', f_score)
        results.set_value(num+1, 'precision', precision)
        results.set_value(num+1, 'recall', recall)
        num += 1
    print('Aggregated f_score:', results['f_score'].mean())
    print('Aggregated precision:', results['precision'].mean())
    print('Aggregated recall:', results['recall'].mean())
    return results

In [13]:
# Perform prediction
y_pred = pipeline_rf.predict(X_test)


In [15]:
category_names = y_test.columns

In [16]:
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               request       0.79      0.43      0.56      1092
                 offer       0.00      0.00      0.00        28
           aid_related       0.77      0.59      0.67      2777
          medical_help       0.67      0.11      0.19       589
      medical_products       0.65      0.08      0.14       369
     search_and_rescue       0.50      0.04      0.07       192
              security       0.33      0.01      0.02       124
              military       0.68      0.08      0.15       230
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.24      0.37       435
                  food       0.88      0.49      0.63       763
               shelter       0.81      0.28      0.41       601
              clothing       0.85      0.10      0.18       108
                 money       0.62      0.03      0.06       154
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [20]:
# Printing the classification report for each output category
results = print_results(y_test, y_pred)
results

Aggregated f_score: 0.933606033126
Aggregated precision: 0.935211610102
Aggregated recall: 0.946837264048


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarn

,Category,f_score,precision,recall
1,request,0.871239,0.877353,0.885566
2,offer,0.993596,0.991474,0.995728
3,aid_related,0.743649,0.752329,0.750076
4,medical_help,0.886218,0.896697,0.915166
5,medical_products,0.92489,0.930936,0.945682
6,search_and_rescue,0.95825,0.957894,0.970705
7,security,0.971931,0.968966,0.980928
8,military,0.953556,0.957523,0.966433
9,child_alone,1,1,1
10,water,0.932332,0.941694,0.946597


### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
# Show parameters for the pipline
pipeline_rf.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f18fc639598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [22]:
# Using grid search
# Create Grid search parameters for Random Forest Classifier   
parameters_rf = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[1, 2, 5, 10],
              'clf__estimator__n_estimators': [10, 20]}

cv_rf = GridSearchCV(pipeline_rf, param_grid = parameters_rf)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
cv_rf.fit(X_train, y_train)
y_pred = cv_rf.predict(X_test)
results2 = print_results(y_test, y_pred)
results2

KeyboardInterrupt: 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# testing a pure decision tree classifier
estimator_ada_boost = MultiOutputClassifier(AdaBoostClassifier())

pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  estimator_ada_boost)
    ])

In [ ]:
pipeline_ada.get_params()

In [ ]:
parameters_ada = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]
}

cv_ada = GridSearchCV(pipeline_ada, param_grid = parameters_ada)

cv_ada.fit(X_train, y_train)
y_pred_ada = cv_ada.predict(X_test)
results3 = print_results(y_test, y_pred_ada)
results3

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.